# Map Reduce Oefening - Schaken

**Deadline:** 05/03/2023 om 23u59

In deze oefening gaan we werken met een dataset dat informatie bevat over een groot aantal schaakspelletjes.
Deze dataset kan [hier](https://www.kaggle.com/datasnaek/chess) gevonden worden.
Om te beginnen, download deze file en upload hem naar je distributed file systeem onder het path: **Oefingen/Mapreduce**.
Voorzie ook code die deze folder reset naar een lege folder om geen naam-conflicten te hebben.

In [16]:
import pydoop.hdfs as hdfs

In [17]:
# make links with local and hdfs file systems
localFS = hdfs.hdfs(host='')
client = hdfs.hdfs(host='localhost', port=9000)

# if directory does not exists - make directory
if not client.exists('/Oefeningen/Mapreduce'):
    client.create_directory('/Oefeningen/Mapreduce')
client.set_working_directory('/Oefeningen/Mapreduce')
print(client.working_directory())

# do some cleaning in case anything else than input.txt is present
for f in client.list_directory("."):
    if not f["name"].endswith("games.csv"):
        client.delete(f["name"], True) # True om aan te geven dat folders ook verwijderd moeten worden

2023-03-01 12:09:18,627 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/Oefeningen/Mapreduce


In [18]:
localFS.copy("games.csv", client, "games.csv") # upload file to hdfs

0

# Map Reduce applicaties

Nu dat de dataset geupload is kunnen we map-reduce applicaties schrijven om deze dataset te verwerken. 
Deze oefeningen bestaan steeds uit twee cellen:
* Een eerste cel met de nodige python code voor de map-reduce applicatie. Sla deze code op in een file met het correcte oefeningennummer (bvb: oefening_1.py) door gebruik te maken van de "%%file" tag in de notebookcellen.
* Een tweede cel die de map-reduce applicatie uitvoert op de cluster en het correcte bestand uitleest. Commando's uitvoeren naar de commandline/terminal gebeurt door een uitroepingsteken vooraan te plaatsen (!pydoop submit ...)

## Aantal rijen

De eerste gevraagde map-reduce applicatie telt het aantal rijen.
Schrijf deze nu hieronder en beantwoord de vragen na de code:

In [19]:
#!echo bigdata | sudo -S pip install mrjob

In [20]:
%%writefile oefening1.py
from mrjob.job import MRJob

class MrLineCount(MRJob):
    def mapper(self, _, line):
        yield "Aantal rijen: ", 1
        
    def reducer(self, key, values):
        yield key, sum(values)
        
if __name__ == '__main__':
    MrLineCount.run()

Overwriting oefening1.py


In [21]:
# commando voor uitvoeren van de applicatie
!python oefening1.py -r hadoop hdfs:////Oefeningen/Mapreduce/games.csv > output_oefening1.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /home/bigdata/hadoop/bin...
Found hadoop binary: /home/bigdata/hadoop/bin/hadoop
Using Hadoop version 3.3.0
Looking for Hadoop streaming jar in /home/bigdata/hadoop...
Found Hadoop streaming jar: /home/bigdata/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
Creating temp directory /tmp/oefening1.bigdata.20230301.110919.766313
uploading working dir files to hdfs:///user/bigdata/tmp/mrjob/oefening1.bigdata.20230301.110919.766313/files/wd...
Copying other local files to hdfs:///user/bigdata/tmp/mrjob/oefening1.bigdata.20230301.110919.766313/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar3264871025194378587/] [] /tmp/streamjob10242601191932769874.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/job_1677

In [22]:
cat output_oefening1.txt

"Aantal rijen: "	20059


**Vragen:**

- Hoeveel rijen heb je gevonden in de dataset?
- Hoeveel spelletjes zitten er echt in de dataset (kijk hiervoor naar de link van de dataset)?
- Zijn deze waarden gelijk? Indien nee, wat is de oorzaak van het verschil? Hoe zou je dit kunnen oplossen?

**Antwoord:**

Volgens de MapReduce zijn er 20 059 rijen.
Volgens de dataset zijn er 20 058 spellen.
Het verschil is dus die ene rij, namelijk de eerste rij met de titels van de kolommen.
De eerste rij begint met "id,". Deze zou je dus kunnen negeren.

Indien je waarden niet overeenkomen, maak een kopie van de vorige applicatie en pas het aan zodat het aantal spelletjes in de dataset correct geteld wordt.

In [23]:
%%writefile oefening1_2.py
# aangepast map-reduce applicatie om het correcte aantal te bekomen.
from mrjob.job import MRJob

class MrLineCount2(MRJob):
    def mapper(self, _, line):
        if line.startswith('id,'):
            return
        yield "Aantal rijen (zonder kolomnamen): ", 1
        
    def reducer(self, key, values):
        yield key, sum(values)
        
if __name__ == '__main__':
    MrLineCount2.run()

Overwriting oefening1_2.py


In [24]:
# commando voor uitvoeren van de applicatie
!python oefening1_2.py -r hadoop hdfs:////Oefeningen/Mapreduce/games.csv > output_oefening1_2.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /home/bigdata/hadoop/bin...
Found hadoop binary: /home/bigdata/hadoop/bin/hadoop
Using Hadoop version 3.3.0
Looking for Hadoop streaming jar in /home/bigdata/hadoop...
Found Hadoop streaming jar: /home/bigdata/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
Creating temp directory /tmp/oefening1_2.bigdata.20230301.110952.015106
uploading working dir files to hdfs:///user/bigdata/tmp/mrjob/oefening1_2.bigdata.20230301.110952.015106/files/wd...
Copying other local files to hdfs:///user/bigdata/tmp/mrjob/oefening1_2.bigdata.20230301.110952.015106/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar9832145311285857253/] [] /tmp/streamjob469524896235871657.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/job_

In [25]:
cat output_oefening1_2.txt

"Aantal rijen (zonder kolomnamen): "	20058


## Aantal kolommen

Naast het aantal rijen kunnen we ook het aantal kolommen bepalen. Aangezien het mogelijk zou zijn dat verschillende rijen een verschillend aantal kolommen hebben is het nodig om hier het maximum aantal kolommen in de verschillende rijen te bepalen.

In [4]:
%%writefile oefening2.py
from mrjob.job import MRJob

class CountColumns(MRJob):

    def mapper(self, _, line):
        fields = line.split(',')
        yield "aantal_kolommen", len(fields)
        
    def reducer(self, key, values):
        yield key, max(values)
if __name__ == '__main__':
    CountColumns.run()

Overwriting oefening2.py


In [5]:
# commando voor uitvoeren van de applicatie
!python oefening2.py -r hadoop hdfs:////Oefeningen/Mapreduce/games.csv > output_oefening2.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /home/bigdata/hadoop/bin...
Found hadoop binary: /home/bigdata/hadoop/bin/hadoop
Using Hadoop version 3.3.0
Looking for Hadoop streaming jar in /home/bigdata/hadoop...
Found Hadoop streaming jar: /home/bigdata/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
Creating temp directory /tmp/oefening2.bigdata.20230301.220909.513310
uploading working dir files to hdfs:///user/bigdata/tmp/mrjob/oefening2.bigdata.20230301.220909.513310/files/wd...
Copying other local files to hdfs:///user/bigdata/tmp/mrjob/oefening2.bigdata.20230301.220909.513310/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar18205885377881284026/] [] /tmp/streamjob8879192376925364439.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/job_1677

In [6]:
cat output_oefening2.txt

"aantal_kolommen"	16


**Vragen:**

- Hoeveel kolommen heb je gevonden in de dataset?
- Hoeveel kolommen zitten er echt in de dataset (kijk hiervoor naar de link van de dataset)?
- Zijn deze waarden gelijk? Indien nee, wat is de oorzaak van het verschil? Hoe zou je dit kunnen oplossen?

**Antwoord:**

Er zijn in elke record 16 kolommen. Dit komt overeen met het aantal kolommen in de header (titels van kolommen) en op Kaggle. Het is dus in orde.

Indien je waarden niet overeenkomen, maak een kopie van de vorige applicatie en pas het aan zodat het aantal spelletjes in de dataset correct geteld wordt.

In [29]:
# aangepast map-reduce applicatie om het correcte aantal te bekomen.

## Aantal uitkomsten

Er zijn vier verschillende uitkomsten mogelijk bij een schaakspel: mate (schaakmat), resign (opgave), draw(gelijkspel), out of time (tijd overschreden).
Maak nu een map-reduce applicatie dat telt hoe vaak elk van deze uitkomsten plaatsvindt

In [30]:
%%writefile oefening3.py
# map-reduce applicatie om het aantal keer dat de verschillende uitkomsten voorkomen te berekenen
from mrjob.job import MRJob
import csv

class WordCount(MRJob):

    def mapper(self, _, line):
        row = next(csv.reader([line]))
        if len(row) > 1:  # Andere methode dan anders maar om ervoor te zorgen dat hij alleen count in de kolom van victory_status, omdat anders de counts correct waren.
            text = row[5]
            words = text.split()
            for word in words:
                if word.lower() in ['draw', 'resign', 'outoftime', 'mate']:
                    yield (word.lower(), 1)

    def combiner(self, word, counts):
        yield (word, sum(counts))

    def reducer(self, word, counts):
        yield (word, sum(counts))

if __name__ == '__main__':
    WordCount.run()      

Overwriting oefening3.py


In [31]:
# commando voor uitvoeren van de applicatie
!python oefening3.py -r hadoop hdfs:////Oefeningen/Mapreduce/games.csv > output_oefening3.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /home/bigdata/hadoop/bin...
Found hadoop binary: /home/bigdata/hadoop/bin/hadoop
Using Hadoop version 3.3.0
Looking for Hadoop streaming jar in /home/bigdata/hadoop...
Found Hadoop streaming jar: /home/bigdata/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
Creating temp directory /tmp/oefening3.bigdata.20230301.111053.242441
uploading working dir files to hdfs:///user/bigdata/tmp/mrjob/oefening3.bigdata.20230301.111053.242441/files/wd...
Copying other local files to hdfs:///user/bigdata/tmp/mrjob/oefening3.bigdata.20230301.111053.242441/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar1588086365864755549/] [] /tmp/streamjob5972805352601566205.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/job_16776

In [32]:
cat output_oefening3.txt

"draw"	906
"mate"	6325
"outoftime"	1680
"resign"	11147


**Vragen:**

* Hoeveel komt elke waarde voor?

**Antwoord:**

Draw komt 906 keer voor.
Mate komt 6325 keer voor.
Outoftime komt 1680 keer voor.
Resign komt 11147 keer voor.
Met een totaal van 20058 items = aantal records in de csv.

## Welke kleur wint het meest

Schrijf nu een map-reduce applicatie dat telt hoevaak elk kleur wint. Zorg er ook voor dat je naast de totaal aantal gewonnen spelletjes, ook enkel de spelletjes telt waar de rating van de spelers niet sterk verschilt (max 100 verschil).

In [33]:
%%writefile oefening4_1.py
# map-reduce applicatie om de winstkansen van een kleur te bepalen. (ALLE GAMES)
from mrjob.job import MRJob
import csv

class WinCountAll(MRJob):

    def mapper(self, _, line):
        fields = line.split(',')
        words = fields[6].split()
        
        words_to_count = ['black', 'white', 'draw']
        
        for word in words_to_count:
            if word in words:
                yield (word, 1)         

    def reducer(self, key, values):
        # Sum the counts for each word
        yield (key, sum(values))

if __name__ == '__main__':
    WinCountAll.run()     

Overwriting oefening4_1.py


In [34]:
# commando voor uitvoeren van de applicatie
!python oefening4_1.py -r hadoop hdfs:////Oefeningen/Mapreduce/games.csv > output_oefening4_1.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /home/bigdata/hadoop/bin...
Found hadoop binary: /home/bigdata/hadoop/bin/hadoop
Using Hadoop version 3.3.0
Looking for Hadoop streaming jar in /home/bigdata/hadoop...
Found Hadoop streaming jar: /home/bigdata/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
Creating temp directory /tmp/oefening4_1.bigdata.20230301.111126.102143
uploading working dir files to hdfs:///user/bigdata/tmp/mrjob/oefening4_1.bigdata.20230301.111126.102143/files/wd...
Copying other local files to hdfs:///user/bigdata/tmp/mrjob/oefening4_1.bigdata.20230301.111126.102143/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar18320228680212727941/] [] /tmp/streamjob11960526563916170843.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/j

In [35]:
cat output_oefening4_1.txt

"black"	9107
"draw"	950
"white"	10001


In [36]:
%%writefile oefening4_2.py
# map-reduce applicatie om de winstkansen van een kleur te bepalen. (ALLEEN ALS VERSCHIL KLEINER IS DAN 100)
from mrjob.job import MRJob
import csv

class WinCountExtra(MRJob):
    def mapper(self, _, line):
        fields = line.split(',')
        
        # tr/except return om fout te voorkomen
        try:
            white_rating = int(fields[9])
            black_rating = int(fields[11])
            if abs(white_rating - black_rating) <= 100:
                column7 = fields[6]
                words = column7.split()
                
                words_to_count = ['black', 'white', 'draw']
                
                for word in words_to_count:
                    if word in words:
                        yield (word, 1)
        except:
            return
        


    def reducer(self, key, values):
        yield (key, sum(values))
        
if __name__ == '__main__':
    WinCountExtra.run()             

Overwriting oefening4_2.py


In [37]:
# commando voor uitvoeren van de applicatie
!python oefening4_2.py -r hadoop hdfs:////Oefeningen/Mapreduce/games.csv > output_oefening4_2.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /home/bigdata/hadoop/bin...
Found hadoop binary: /home/bigdata/hadoop/bin/hadoop
Using Hadoop version 3.3.0
Looking for Hadoop streaming jar in /home/bigdata/hadoop...
Found Hadoop streaming jar: /home/bigdata/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
Creating temp directory /tmp/oefening4_2.bigdata.20230301.111158.639862
uploading working dir files to hdfs:///user/bigdata/tmp/mrjob/oefening4_2.bigdata.20230301.111158.639862/files/wd...
Copying other local files to hdfs:///user/bigdata/tmp/mrjob/oefening4_2.bigdata.20230301.111158.639862/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar17160117306632498875/] [] /tmp/streamjob939095438643322709.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/job

In [38]:
cat output_oefening4_2.txt

"black"	4168
"draw"	490
"white"	4504


**Vragen:**

* Welke kleur heeft het meeste gewonnen?
* Is hetzelfde vast te stellen wanneer we enkel kijken naar gelijkwaardige spelers? (Bereken hiervoor de percentages!)

**Antwoord:**

Alle spelletjes:
- Zwart wint 9107 keer op 20058. (=45,404%)
- Wit wint 10001 keer op 20058. (=49,860%)
- Gelijkspel 950 keer op 20058. (=4,736%)

Spelletjes met gelijkwaardige spelers:
- Zwart wint 4168 keer op 9162. (=45,492%)
- Wit wint 4504 keer op 9162. (=49,160%)
- Gelijkspel 490 keer op 9162. (=5,348%)

Conclusie: er zijn kleine verschillen van maximaal 1% dus we kunnen zeggen dat we hetzelfde vaststellen bij alle spelletjes vs spelletjes met alleen gelijkwaardige spelers.

## Verschillende openingen

Een belangrijk onderdeel van schaakspelletjes is de openingset die gebruikt wordt. Schrijf nu een map-reduce applicatie dat telt hoevaak elke openingsset resulteert in een winst of verlies voor de witte speler. 
Emit hierbij in de mapping fase enkel key-value pairs waar de keys de namen van de openingszetten zijn.
Filteren tussen winst en verlies moet dan in de reduce fase gebeuren.

In [39]:
%%writefile oefening5.py
# winst of verlies van openingszetten

from mrjob.job import MRJob
import csv

from mrjob.job import MRJob

class RecordCount(MRJob):

    def mapper(self, _, line):
        fields = line.split(',')
        openingset = fields[14]
        winner = fields[6]
        
        if line.startswith('id,'):
            return
        yield openingset, winner
        
    def reducer(self, key, values):
        wins = 0
        losses = 0
        draws = 0

        for value in values:
            if value == 'white':
                wins += 1
            elif value == 'black':
                losses += 1
            elif value == 'draw':
                draws += 1

        yield key, (wins, losses, draws)        

if __name__ == '__main__':
    RecordCount.run()

Overwriting oefening5.py


In [40]:
# commando voor uitvoeren van de applicatie
!python oefening5.py -r hadoop hdfs:////Oefeningen/Mapreduce/games.csv > output_oefening5.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /home/bigdata/hadoop/bin...
Found hadoop binary: /home/bigdata/hadoop/bin/hadoop
Using Hadoop version 3.3.0
Looking for Hadoop streaming jar in /home/bigdata/hadoop...
Found Hadoop streaming jar: /home/bigdata/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
Creating temp directory /tmp/oefening5.bigdata.20230301.111230.480455
uploading working dir files to hdfs:///user/bigdata/tmp/mrjob/oefening5.bigdata.20230301.111230.480455/files/wd...
Copying other local files to hdfs:///user/bigdata/tmp/mrjob/oefening5.bigdata.20230301.111230.480455/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar14888274783995662160/] [] /tmp/streamjob1530801293548106478.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/job_1677

In [41]:
cat output_oefening5.txt

"Alekhine Defense #2"	[1, 1, 1]
"Alekhine Defense #3"	[0, 1, 0]
"Alekhine Defense"	[23, 21, 2]
"Alekhine Defense: Balogh Variation"	[0, 1, 0]
"Alekhine Defense: Brooklyn Variation"	[2, 0, 0]
"Alekhine Defense: Exchange Variation"	[3, 1, 0]
"Alekhine Defense: Four Pawns Attack |  6...Nc6"	[1, 0, 0]
"Alekhine Defense: Four Pawns Attack |  Fianchetto Variation"	[1, 0, 0]
"Alekhine Defense: Four Pawns Attack |  Main Line"	[1, 0, 0]
"Alekhine Defense: Four Pawns Attack |  Trifunovic Variation"	[2, 0, 0]
"Alekhine Defense: Four Pawns Attack"	[1, 1, 0]
"Alekhine Defense: John Tracy Gambit"	[2, 2, 0]
"Alekhine Defense: Krejcik Variation"	[1, 1, 0]
"Alekhine Defense: Maroczy Variation"	[10, 8, 1]
"Alekhine Defense: Modern Variation |  Larsen Variation"	[2, 1, 0]
"Alekhine Defense: Modern Variation |  Larsen-Haakert Variation"	[0, 1, 0]
"Alekhine Defense: Modern Variation |  Main Line"	[3, 2, 0]
"Alekhine Defense: Mokele Mbembe |  Vavra Defense"	[4, 0, 0]
"Alekhine Defense: Mokele Mbembe"	[9, 2,

De output van bovenstaande applicatie zou een groot aantal rijen moeten bevatten. Download het bestand en bekijk het aantal rijen in dit bestand. 

**Hoeveel verschillende openingszetten zijn er aanwezig in de dataset?**

Er zijn 1477 lijnen in het output bestand. 
Template: "naam": [win count, loss count, draw count]

## Verificatie van de kolom met het aantal zetten

In de dataset zijn twee manieren aanwezig om het aantal zetten in een schaakspel te bekomen.
* Er is een turns kolom dat het totaal aantal beurten vast heeft
* Er is ook een moves kolom dat alle uitgevoerde zetten bevat (gescheidden door een spatie)

Kijk of deze waarden voor alle rijen overeenkomt en tel de volgende waarden:
* Totaal aantal schaakspelletjes
* Aantal waar de kolommen overeenkomen
* Aantal waar de kolommen niet overeenkomen.

Zorg ervoor dat deze drie waarden aanwezig zijn in het uiteindelijke resultaat.

In [42]:
%%file oefening_zetten.py

from mrjob.job import MRJob
import csv

class WinCountExtra(MRJob):
    def mapper(self, _, line):
        if line.startswith('id,'):
            return
        fields = line.split(',')
        yield ("totaal", 1)
        

        moves_column = str(fields[12])
        turns_column = int(fields[4])

        zetten_split_count = len(moves_column.split())
        if turns_column == zetten_split_count:
            yield ("overeenkomend", 1)
        if turns_column != zetten_split_count:
            yield ("nietovereenkomend", 1)   
         
    def reducer(self, key, values):
        yield (key, sum(values))
        
if __name__ == '__main__':
    WinCountExtra.run()   

Overwriting oefening_zetten.py


In [43]:
# commando voor uitvoeren van de applicatie
!python oefening_zetten.py -r hadoop hdfs:////Oefeningen/Mapreduce/games.csv > output_oefening_zetten.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /home/bigdata/hadoop/bin...
Found hadoop binary: /home/bigdata/hadoop/bin/hadoop
Using Hadoop version 3.3.0
Looking for Hadoop streaming jar in /home/bigdata/hadoop...
Found Hadoop streaming jar: /home/bigdata/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
Creating temp directory /tmp/oefening_zetten.bigdata.20230301.111303.252351
uploading working dir files to hdfs:///user/bigdata/tmp/mrjob/oefening_zetten.bigdata.20230301.111303.252351/files/wd...
Copying other local files to hdfs:///user/bigdata/tmp/mrjob/oefening_zetten.bigdata.20230301.111303.252351/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar18107195690825420138/] [] /tmp/streamjob1389034145496353604.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata

**Plaats hieronder de bekomen output voor dit commando:**

In [44]:
cat output_oefening_zetten.txt

"overeenkomend"	20058
"totaal"	20058


## Aantal verschillende spelers en aantal verschillende openingen

Schrijf nu een map-reduce applicatie dat zowel het aantal verschillende spelers als het aantal verschillende openingen in de dataset telt. Tip: een set in python bevat geen duplicaten

In [45]:
%%writefile oefening_spelers_openingen.py
# aantal spelers en openingen

from mrjob.job import MRJob

class CountPlayersAndOpenings(MRJob):
        
    def mapper(self, _, line):
        if line.startswith('id,'):
            return
        fields = line.split(',')
        
        player1 = str(fields[8]).strip().lower()
        player2 = str(fields[10]).strip().lower()
        opening = str(fields[14]).strip().lower()
        
        yield "players", player1
        yield "players", player2
        yield "openings", opening
                    
    def reducer(self, key, values):
        unique_values = {}
        
        for value in values:
            unique_values[value] = True
        
        count = len(unique_values)
        
        yield key, count
        
        
if __name__ == '__main__':
    CountPlayersAndOpenings.run()

Overwriting oefening_spelers_openingen.py


In [46]:
# commando voor uitvoeren van de applicatie
!python oefening_spelers_openingen.py -r hadoop hdfs:////Oefeningen/Mapreduce/games.csv > output_oefening_spelers_openingen.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /home/bigdata/hadoop/bin...
Found hadoop binary: /home/bigdata/hadoop/bin/hadoop
Using Hadoop version 3.3.0
Looking for Hadoop streaming jar in /home/bigdata/hadoop...
Found Hadoop streaming jar: /home/bigdata/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
Creating temp directory /tmp/oefening_spelers_openingen.bigdata.20230301.111335.142285
uploading working dir files to hdfs:///user/bigdata/tmp/mrjob/oefening_spelers_openingen.bigdata.20230301.111335.142285/files/wd...
Copying other local files to hdfs:///user/bigdata/tmp/mrjob/oefening_spelers_openingen.bigdata.20230301.111335.142285/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar11938558430078460629/] [] /tmp/streamjob2437637222829304818.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Disabling Erasure Coding for path:

**Plaats hieronder de bekomen output voor dit commando:**

In [47]:
cat output_oefening_spelers_openingen.txt

"openings"	1477
"players"	15635


## verscheidene statistieken in 1 applicatie

Maak nu een map-reduce applicatie dat de volgende zaken in 1 keer berekend:
* Het aantal minuten dat een schaakspel geduurd heeft (de tijden aanwezig in de dataset zijn in [Unix-format](https://en.wikipedia.org/wiki/Unix_time))
* Het minimum, maximum en gemiddeld aantal beurten
* Percentage rated
* Het minimum, maximum en gemiddeld aantal keer dat er schaak voorkwam in een spel dat eindige in schaakmat. (Schaak wordt aangegeven door een "+" in de moves kolom.)

In [48]:
%%writefile oefening_verschillende_statistieken.py

from mrjob.job import MRJob
import statistics

class VerschillendeStatistiekenJob(MRJob):
    def mapper(self, _, line):
        if line.startswith('id,'):
            return
        
        fields = line.split(',')

        # AANTAL MINUTEN: MEDIAAN
        # Tijden van 0 of 10000000 zijn berekeningen waar de begin en eind date gelijk zijn of fout. 
        # Daarom yield ik ze niet om een correctere mediaan vervolgens te hebben.

        difference_in_time = float(fields[3]) - float(fields[2])
        
        if(difference_in_time != 0) and (difference_in_time != 10000000):
            aantal_min = difference_in_time / 1000 / 60
            yield "aantal_minuten", aantal_min
            
        # RATED
        if(str(fields[1]).lower() == "true"):
            yield "rated", 1
        else:
            yield "rated", 0
            
        # AANTAL BEURTEN
        yield "aantal_beurten", int(fields[4])
            
        # AANTAL KEER DAT SCHAAK VOORKWAM IN EEN SPEL DAT EINDIGDE IN SCHAAKMAT
        if(str(fields[5]).lower() == "mate"):
            yield "aantal_schaak", int(fields[12].count("+"))
            

    def reducer(self, key, values):
        valuesfix = list(values)
        # AANTAL MINUTEN: MEDIAAN
        if(key == "aantal_minuten"):
            yield "mediaan_minuten", statistics.median(valuesfix)
            
        # RATED: PERCENTAGE
        elif(key == "rated"):
            yield "rated_%", sum(valuesfix) / len(valuesfix) * 100
            
        # BEURTEN EN SCHAAK: MIN, MAX EN GEM
        else:
            yield "minimum_" + key, min(valuesfix)
            yield "maximum_" + key, max(valuesfix)
            yield "average_" + key, sum(valuesfix) / len(valuesfix)

if __name__ == '__main__':
    VerschillendeStatistiekenJob.run()      

Overwriting oefening_verschillende_statistieken.py


In [49]:
# commando voor uitvoeren van de applicatie
!python oefening_verschillende_statistieken.py -r hadoop hdfs:////Oefeningen/Mapreduce/games.csv > output_oefening_verschillende_statistieken.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /home/bigdata/hadoop/bin...
Found hadoop binary: /home/bigdata/hadoop/bin/hadoop
Using Hadoop version 3.3.0
Looking for Hadoop streaming jar in /home/bigdata/hadoop...
Found Hadoop streaming jar: /home/bigdata/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
Creating temp directory /tmp/oefening_verschillende_statistieken.bigdata.20230301.111406.920443
uploading working dir files to hdfs:///user/bigdata/tmp/mrjob/oefening_verschillende_statistieken.bigdata.20230301.111406.920443/files/wd...
Copying other local files to hdfs:///user/bigdata/tmp/mrjob/oefening_verschillende_statistieken.bigdata.20230301.111406.920443/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar14051417289884979916/] [] /tmp/streamjob15882591260106192213.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Disabl

**Plaats hieronder de bekomen output voor dit commando:**

In [50]:
cat output_oefening_verschillende_statistieken.txt

"minimum_aantal_beurten"	1
"maximum_aantal_beurten"	349
"average_aantal_beurten"	60.46599860404826
"mediaan_minuten"	11.02435
"minimum_aantal_schaak"	0
"maximum_aantal_schaak"	46
"average_aantal_schaak"	5.875573122529644
"rated_%"	80.54142985342507
